# Working with Langchain

In [1]:
# pip install langchain langchain-experimental langchain-openai

### Chatmodels, Pormpt Templates and Parsers

In [2]:
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(api_version="2024-12-01-preview",model='telcogpt')

prompt = "Tell me a joke about mouse"
model.invoke(prompt)

AIMessage(content='Why did the mouse stay home from school?\n\nBecause it was afraid of the cat-astrophe!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 13, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_7a53abb7a2', 'id': 'chatcmpl-BWxmfEwTiShwuqV4jR0cuLVvVReK9', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': Fals

In [3]:
from langchain_core.prompts import ChatPromptTemplate
system_template = "Translate the following into {language}"

prompt = ChatPromptTemplate([("system",system_template),("user","{text}")])

prompt

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='Translate the following into {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [4]:
prompt.input_variables

['language', 'text']

In [5]:
prompt.invoke({"language":"hindi","text":"HOW ARE YOU?"})

ChatPromptValue(messages=[SystemMessage(content='Translate the following into hindi', additional_kwargs={}, response_metadata={}), HumanMessage(content='HOW ARE YOU?', additional_kwargs={}, response_metadata={})])

In [6]:
# PARSERS
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
final_prompt = prompt.invoke({"language":"hindi","text":"HOW ARE YOU?"})
op = model.invoke(final_prompt)
op

AIMessage(content='आप कैसे हैं?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 20, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_7a53abb7a2', 'id': 'chatcmpl-BWxneoCBMO0TQehMBNFGUo88bswVJ', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 

In [7]:
parser.invoke(op)

'आप कैसे हैं?'

## Chain
- a static sequence of execution involving multiple components such as LLMs, parsers, prompts, tools, loaders etc.
- can be used to automate a rule based, linear and non linear workflows involving LLMs, tools

In [9]:
translation_chain = prompt | model | parser
translation_chain.invoke({"language":'tamil','text':'Thank You'})

'நன்றி'

In [10]:
def format_output(text):
    return {"Translation":text}

In [11]:
translation_chain = prompt | model | parser | format_output
translation_chain.invoke({"language":'tamil','text':'Thank You'})

{'Translation': 'நன்றி'}

#### Code Chain

In [14]:
generate_prompt = ChatPromptTemplate.from_template("Write a python code for {task}, Only provide python code, NO any additional text")

chain1 = generate_prompt | model | parser

op = chain1.invoke({"task":"fibonacci series"})
print(op)

```python
def fibonacci(n):
    fib_series = []
    a, b = 0, 1
    for _ in range(n):
        fib_series.append(a)
        a, b = b, a + b
    return fib_series

n = int(input("Enter the number of terms: "))
print(fibonacci(n))
```


In [15]:
analyze_prompt = ChatPromptTemplate.from_template("Analyze the provided code and calculate time complexity {code}")
chain2 = analyze_prompt | model | parser

In [ ]:
def patch_response(code):
    print(code)
    print ("-"*20)
    return {"code":code}


final_chain = chain1 | patch_response | chain2

op = final_chain.invoke({"task":'fibonacci series'})
print(op)

```python
def fibonacci(n):
    fib_series = [0, 1]
    for i in range(2, n):
        next_value = fib_series[-1] + fib_series[-2]
        fib_series.append(next_value)
    return fib_series[:n]

# Example usage
n = 10
print(fibonacci(n))
```
--------------------


"Let's analyze the provided `fibonacci` function step-by-step to determine its time complexity.\n\n### Code Analysis\nThe function `fibonacci(n)` generates the Fibonacci series up to the `n`-th term. Here's a breakdown of what the function does:\n\n1. **Initialization**: It starts by creating a list `fib_series` containing the first two Fibonacci numbers: `[0, 1]`.\n\n2. **Loop to Calculate Fibonacci Numbers**: The `for` loop runs from `2` to `n-1` (inclusive), meaning it executes `n-2` iterations (for an input of `n`, this means it calculates `n-2` additional Fibonacci numbers beyond the initial two).\n\n3. **Fibonacci Calculation**:\n   - In each iteration of the loop, it calculates the next Fibonacci value by summing the last two values in the `fib_series` list (i.e., `fib_series[-1]` and `fib_series[-2]`).\n   - After calculating the next Fibonacci value, it appends this value to the `fib_series` list.\n\n4. **Slicing the List**: Finally, it returns a slice of the first `n` Fibonac

In [ ]:
final_chain = generate_prompt | model | parser | patch_response | analyze_prompt | model | parser
op = final_chain.invoke({"task":"testing whether the number input is odd or even"})
print(op)

```python
number = int(input("Enter a number: "))
if number % 2 == 0:
    print("Even")
else:
    print("Odd")
```
--------------------
